In [ ]:
# ======================================================================================
# Geochemical Multivariate Analysis
# PCA, Factor Analysis, Correlation and Hierarchical Clustering
# SPSS-equivalent workflow implemented in Python
#
# Author: Gabby Roveratti
# ======================================================================================

# =====================================
# Libraries
# =====================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import squareform
from scipy.stats import pearsonr

from factor_analyzer import (
    FactorAnalyzer,
    calculate_kmo,
    calculate_bartlett_sphericity
)

import warnings
warnings.filterwarnings("ignore")

# =====================================
# Plot settings
# =====================================

plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.max_open_warning"] = 50
plt.rcParams["font.size"] = 10
sns.set_style("whitegrid")

# =====================================
# Data loading
# =====================================

DATA_PATH = "data/example_dataset.xlsx"
SHEET_NAME = "Sheet1"

df = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)

# Replace common non-numeric strings
df.replace(["-", "n.d.", "n/a", "null", " ", ""], np.nan, inplace=True)

# =====================================
# Oxide selection
# =====================================

oxides = [
    "K (%)", "U (ppm)", "Th (ppm)", "SiO2", "MgO", "Fe2O3", "SO3",
    "Al2O3", "CaO", "K2O", "Na2O", "TiO2", "P2O5",
    "SrO", "MnO", "CuO", "Cl"
]

sample_names = df["Sample"].astype(str).values
lithologies = df["Lithology"].astype(str).values

df_oxides = df[oxides].apply(pd.to_numeric, errors="coerce")

# =====================================
# Sample completeness
# =====================================

complete_mask = df_oxides.notna().all(axis=1)
incomplete_mask = ~complete_mask

samples_complete = sample_names[complete_mask]
samples_incomplete = sample_names[incomplete_mask]

print("=" * 80)
print("COMPLETE GEOCHEMICAL MULTIVARIATE ANALYSIS")
print("=" * 80)
print(f"Total samples: {len(sample_names)}")
print(f"Complete samples: {len(samples_complete)}")
print(f"Incomplete samples: {len(samples_incomplete)}")

# ======================================================================================
# OPTION 1 — ANALYSIS WITH IMPUTATION (ALL SAMPLES)
# ======================================================================================

print("\n" + "=" * 80)
print("OPTION 1 — ANALYSIS WITH MEDIAN IMPUTATION")
print("=" * 80)

# Imputation
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(df_oxides)
df_imputed = pd.DataFrame(X_imputed, columns=oxides)

# Standardization
scaler_imp = StandardScaler()
X_imp_scaled = scaler_imp.fit_transform(df_imputed)

# KMO and Bartlett
kmo_all_imp, kmo_model_imp = calculate_kmo(X_imp_scaled)
chi2_imp, p_imp = calculate_bartlett_sphericity(X_imp_scaled)

print(f"KMO: {kmo_model_imp:.3f}")
print(f"Bartlett p-value: {p_imp:.5f}")

# =====================================
# Correlation matrix (imputed)
# =====================================

plt.figure(figsize=(14, 12))
corr_imp = df_imputed.corr()
mask = np.triu(np.ones_like(corr_imp, dtype=bool))

sns.heatmap(
    corr_imp,
    mask=mask,
    cmap="RdBu_r",
    center=0,
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5
)

plt.title("Correlation Matrix — Imputed Data")
plt.tight_layout()
plt.show()

# =====================================
# PCA — Imputed data
# =====================================

pca_imp = PCA()
scores_imp = pca_imp.fit_transform(X_imp_scaled)

eigenvalues_imp = pca_imp.explained_variance_
variance_cum_imp = np.cumsum(pca_imp.explained_variance_ratio_) * 100

# Scree plot
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

ax[0].plot(range(1, len(eigenvalues_imp) + 1), eigenvalues_imp, "o-")
ax[0].axhline(1, color="red", linestyle="--")
ax[0].set_title("Scree Plot — Imputed Data")
ax[0].set_xlabel("Principal Component")
ax[0].set_ylabel("Eigenvalue")

ax[1].plot(range(1, len(variance_cum_imp) + 1), variance_cum_imp, "o-")
ax[1].axhline(80, color="red", linestyle="--")
ax[1].set_title("Cumulative Explained Variance (%)")
ax[1].set_xlabel("Principal Component")
ax[1].set_ylabel("Variance (%)")

plt.tight_layout()
plt.show()

# =====================================
# Factor Analysis — Imputed
# =====================================

n_factors_imp = np.sum(eigenvalues_imp > 1)
fa_imp = FactorAnalyzer(n_factors=n_factors_imp, rotation="varimax")
fa_imp.fit(X_imp_scaled)

loadings_imp = pd.DataFrame(
    fa_imp.loadings_,
    index=oxides,
    columns=[f"Factor {i+1}" for i in range(n_factors_imp)]
)

plt.figure(figsize=(10, 12))
sns.heatmap(loadings_imp, cmap="RdBu_r", center=0, annot=True, fmt=".3f")
plt.title("Factor Loadings — Imputed Data (Varimax)")
plt.tight_layout()
plt.show()

# =====================================
# Hierarchical clustering — Samples
# =====================================

corr_samples_imp = np.corrcoef(X_imp_scaled)
dist_samples_imp = 1 - np.abs(corr_samples_imp)
np.fill_diagonal(dist_samples_imp, 0)

Z_samples_imp = linkage(squareform(dist_samples_imp), method="average")

plt.figure(figsize=(16, 8))
dendrogram(
    Z_samples_imp,
    labels=sample_names,
    leaf_rotation=90,
    leaf_font_size=8
)
plt.title("Hierarchical Clustering — Samples (Imputed Data)")
plt.ylabel("Distance (1 − |r|)")
plt.tight_layout()
plt.show()

# ======================================================================================
# OPTION 2 — ANALYSIS WITH COMPLETE SAMPLES ONLY
# ======================================================================================

print("\n" + "=" * 80)
print("OPTION 2 — ANALYSIS WITH COMPLETE SAMPLES ONLY")
print("=" * 80)

df_complete = df_oxides[complete_mask]
X_complete = StandardScaler().fit_transform(df_complete)

kmo_all_c, kmo_model_c = calculate_kmo(X_complete)
chi2_c, p_c = calculate_bartlett_sphericity(X_complete)

print(f"KMO: {kmo_model_c:.3f}")
print(f"Bartlett p-value: {p_c:.5f}")

pca_c = PCA()
scores_c = pca_c.fit_transform(X_complete)

# ======================================================================================
# Final report
# ======================================================================================

print("\n" + "=" * 80)
print("FINAL REPORT")
print("=" * 80)

print("This script reproduces SPSS multivariate procedures using Python.")
print("Option 1 uses median imputation (SPSS-like behavior).")
print("Option 2 uses only complete samples (conservative approach).")

print(f"\nKMO comparison:")
print(f"  Imputed:  {kmo_model_imp:.3f}")
print(f"  Complete: {kmo_model_c:.3f}")

print("\nAnalysis completed successfully.")


In [ ]:
# ======================================================================================
# Geochemical Multivariate Analysis
# PCA, Factor Analysis, Correlation and Hierarchical Clustering
# SPSS-equivalent workflow implemented in Python
#
# Author: Gabby Roveratti
# ======================================================================================

# =====================================
# Libraries
# =====================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

from factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

import warnings
warnings.filterwarnings("ignore")

# =====================================
# Plot settings
# =====================================

sns.set_style("whitegrid")
plt.rcParams["font.size"] = 10

# =====================================
# Output folders
# =====================================

os.makedirs("results/figures", exist_ok=True)

# =====================================
# Data loading
# =====================================

DATA_PATH = "data/example_dataset.xlsx"
SHEET_NAME = "Sheet1"

df = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)

df.replace(["-", "n.d.", "n/a", "null", " ", ""], np.nan, inplace=True)

# =====================================
# Variable selection
# =====================================

oxides = [
    "K (%)", "U (ppm)", "Th (ppm)", "SiO2", "MgO", "Fe2O3", "SO3",
    "Al2O3", "CaO", "K2O", "Na2O", "TiO2", "P2O5",
    "SrO", "MnO", "CuO", "Cl"
]

sample_names = df["Sample"].astype(str).values
lithologies = df["Lithology"].astype(str).values

df_oxides = df[oxides].apply(pd.to_numeric, errors="coerce")

# =====================================
# Sample completeness
# =====================================

complete_mask = df_oxides.notna().all(axis=1)

# ======================================================================================
# FUNCTION — PCA BIPLOT
# ======================================================================================

def plot_pca_biplot(scores, loadings, explained_var, lithology,
                    oxide_names, title, output_path):

    plt.figure(figsize=(10, 8))

    for lith in np.unique(lithology):
        mask = lithology == lith
        plt.scatter(
            scores[mask, 0],
            scores[mask, 1],
            s=45,
            alpha=0.75,
            label=lith
        )

    for i, oxide in enumerate(oxide_names):
        plt.arrow(
            0, 0,
            loadings[i, 0] * 3,
            loadings[i, 1] * 3,
            color="red",
            alpha=0.8,
            head_width=0.05
        )
        plt.text(
            loadings[i, 0] * 3.2,
            loadings[i, 1] * 3.2,
            oxide,
            color="red",
            fontsize=9
        )

    plt.axhline(0, color="gray", linewidth=0.8)
    plt.axvline(0, color="gray", linewidth=0.8)

    plt.xlabel(f"E1 ({explained_var[0]:.1f}%)")
    plt.ylabel(f"E2 ({explained_var[1]:.1f}%)")
    plt.title(title)

    plt.legend(title="Lithology", fontsize=9)
    plt.grid(True, linestyle="--", alpha=0.4)

    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.show()

# ======================================================================================
# OPTION 1 — IMPUTED DATA
# ======================================================================================

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(df_oxides)

scaler = StandardScaler()
X_imp_scaled = scaler.fit_transform(X_imputed)

kmo_all_imp, kmo_model_imp = calculate_kmo(X_imp_scaled)
chi2_imp, p_imp = calculate_bartlett_sphericity(X_imp_scaled)

print("\nIMPUTED DATA")
print(f"KMO: {kmo_model_imp:.3f}")
print(f"Bartlett p-value: {p_imp:.5f}")

# PCA
pca_imp = PCA()
scores_imp = pca_imp.fit_transform(X_imp_scaled)

explained_imp = pca_imp.explained_variance_ratio_ * 100
loadings_imp = pca_imp.components_.T

# Biplot
plot_pca_biplot(
    scores_imp,
    loadings_imp,
    explained_imp,
    lithologies,
    oxides,
    "Biplot E1 x E2 – Imputed Data (by Lithology)",
    "results/figures/biplot_E1_E2_imputed.png"
)

# Dendrogram — samples
corr_samples_imp = np.corrcoef(X_imp_scaled)
dist_samples_imp = 1 - np.abs(corr_samples_imp)
np.fill_diagonal(dist_samples_imp, 0)

Z_samples_imp = linkage(squareform(dist_samples_imp), method="average")

plt.figure(figsize=(16, 8))
dendrogram(
    Z_samples_imp,
    labels=sample_names,
    leaf_rotation=90,
    leaf_font_size=8
)
plt.title("Hierarchical Clustering — Samples (Imputed Data)")
plt.ylabel("Distance (1 − |r|)")
plt.tight_layout()
plt.savefig("results/figures/dendrogram_samples_imputed.png", dpi=300)
plt.show()

# ======================================================================================
# OPTION 2 — COMPLETE DATA ONLY
# ======================================================================================

df_complete = df_oxides[complete_mask]
lith_complete = lithologies[complete_mask]

X_complete = StandardScaler().fit_transform(df_complete)

kmo_all_c, kmo_model_c = calculate_kmo(X_complete)
chi2_c, p_c = calculate_bartlett_sphericity(X_complete)

print("\nCOMPLETE DATA")
print(f"KMO: {kmo_model_c:.3f}")
print(f"Bartlett p-value: {p_c:.5f}")

# PCA
pca_c = PCA()
scores_c = pca_c.fit_transform(X_complete)

explained_c = pca_c.explained_variance_ratio_ * 100
loadings_c = pca_c.components_.T

# Biplot
plot_pca_biplot(
    scores_c,
    loadings_c,
    explained_c,
    lith_complete,
    oxides,
    "Biplot E1 x E2 – Complete Data (by Lithology)",
    "results/figures/biplot_E1_E2_complete.png"
)

print("\nAnalysis completed successfully.")
